In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

%load_ext autoreload
%autoreload

In [2]:
from utils import datasets, metrics
from utils.ensembles import EnsembleCPDModel

import numpy as np

import yaml
import torch
import torch.nn as nn

from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger

from tqdm.notebook import tqdm

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from typing import List, Tuple, Dict
import gc

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
SEED = 0
pl.seed_everything(SEED)

# Fixed evaluation pipeline
(use pre-computed out banks)

In [ ]:
def evaluate_metrics_on_set(
    test_out_bank: List[torch.Tensor],
    test_uncertainties_bank: List[torch.Tensor],
    test_labels_bank: List[torch.Tensor],
    threshold: float = 0.5,
    verbose: bool = True,
    device: str = "cuda",
    uncert_th: float = None,
    margin_list: List[int] = None,
) -> Tuple[int, int, int, int, float, float]:

    FP_delays = []
    delays = []
    covers = []
    TN, FP, FN, TP = (0, 0, 0, 0)

    TN_margin_dict, FP_margin_dict, FN_margin_dict, TP_margin_dict = (
        None,
        None,
        None,
        None,
    )

    if margin_list is not None:
        TN_margin_dict, FP_margin_dict, FN_margin_dict, TP_margin_dict = (
            {},
            {},
            {},
            {},
        )
        for margin in margin_list:
            TN_margin_dict[margin] = 0
            FP_margin_dict[margin] = 0
            FN_margin_dict[margin] = 0
            TP_margin_dict[margin] = 0

    with torch.no_grad():
        for test_out, test_uncertainties, test_labels in zip(
            test_out_bank, test_uncertainties_bank, test_labels_bank
        ):
            if test_uncertainties is not None and uncert_th is not None:
                cropped_outs = (test_out > threshold) & (test_uncertainties < uncert_th)

            else:
                cropped_outs = test_out > threshold

            (
                (tn, fp, fn, tp, FP_delay, delay, cover),
                (tn_margin_dict, fp_margin_dict, fn_margin_dict, tp_margin_dict),
            ) = metrics.calculate_metrics(test_labels, cropped_outs, margin_list)

            TN += tn
            FP += fp
            FN += fn
            TP += tp

            if margin_list is not None:
                for margin in margin_list:
                    TN_margin_dict[margin] += tn_margin_dict[margin]
                    FP_margin_dict[margin] += fp_margin_dict[margin]
                    FN_margin_dict[margin] += fn_margin_dict[margin]
                    TP_margin_dict[margin] += tp_margin_dict[margin]

            FP_delays.append(FP_delay.detach().cpu())
            delays.append(delay.detach().cpu())
            covers.extend(cover)

    mean_FP_delay = torch.cat(FP_delays).float().mean().item()
    mean_delay = torch.cat(delays).float().mean().item()
    mean_cover = np.mean(covers)

    if verbose:
        print(
            "TN: {}, FP: {}, FN: {}, TP: {}, DELAY:{}, FP_DELAY:{}, COVER: {}".format(
                TN, FP, FN, TP, mean_delay, mean_FP_delay, mean_cover
            )
        )

    del FP_delays, delays, covers
    gc.collect()
    if "cuda" in device:
        torch.cuda.empty_cache()

    return (
        (TN, FP, FN, TP, mean_delay, mean_FP_delay, mean_cover),
        (TN_margin_dict, FP_margin_dict, FN_margin_dict, TP_margin_dict),
    )

def evaluation_pipeline(     
    test_out_bank: List[torch.Tensor],
    test_uncertainties_bank: List[torch.Tensor],
    test_labels_bank: List[torch.Tensor],
     
    threshold_list: List[float],
    device: str = "cuda",
    verbose: bool = False,
    model_type: str = "seq2seq",
    subseq_len: int = None,
    scale: int = None,
    uncert_th: float = None,
    q: float = None,
    margin_list: List[int] = None,
    step: int = 1,
    alpha: float = 1.0,
) -> Tuple[Tuple[float], dict, dict]:

    cover_dict = {}
    f1_dict = {}

    if margin_list is not None:
        final_f1_margin_dict = {}

    delay_dict = {}
    fp_delay_dict = {}
    confusion_matrix_dict = {}

    if model_type == "cusum_aggr":
        threshold_list = [0.5]
        if verbose and len(threshold_list) > 1:
            print("No need in threshold list for CUSUM. Take threshold = 0.5.")

    for threshold in tqdm(threshold_list):
        final_f1_margin_dict[threshold] = {}

        (
            (TN, FP, FN, TP, mean_delay, mean_fp_delay, cover),
            (TN_margin_dict, FP_margin_dict, FN_margin_dict, TP_margin_dict),
        ) = evaluate_metrics_on_set(
            test_out_bank=test_out_bank,
            test_uncertainties_bank=test_uncertainties_bank,
            test_labels_bank=test_labels_bank,
            threshold=threshold,
            verbose=False,
            device=device,
            uncert_th=uncert_th,
            margin_list=margin_list,
        )

        confusion_matrix_dict[threshold] = (TN, FP, FN, TP)
        delay_dict[threshold] = mean_delay
        fp_delay_dict[threshold] = mean_fp_delay

        cover_dict[threshold] = cover
        f1_dict[threshold] = metrics.F1_score((TN, FP, FN, TP))

        if margin_list is not None:
            f1_margin_dict = {}
            for margin in margin_list:
                (TN_margin, FP_margin, FN_margin, TP_margin) = (
                    TN_margin_dict[margin],
                    FP_margin_dict[margin],
                    FN_margin_dict[margin],
                    TP_margin_dict[margin],
                )
                f1_margin_dict[margin] = metrics.F1_score(
                    (TN_margin, FP_margin, FN_margin, TP_margin)
                )
            final_f1_margin_dict[threshold] = f1_margin_dict

    # fix dict structure
    if margin_list is not None:
        final_f1_margin_dict_fixed = {}
        for margin in margin_list:
            f1_scores_for_margin_dict = {}
            for threshold in threshold_list:
                f1_scores_for_margin_dict[threshold] = final_f1_margin_dict[threshold][
                    margin
                ]
            final_f1_margin_dict_fixed[margin] = f1_scores_for_margin_dict

    if model_type == "cusum_aggr":
        auc = None
    else:
        auc = metrics.area_under_graph(list(delay_dict.values()), list(fp_delay_dict.values()))

    # Conf matrix and F1
    best_th_f1 = max(f1_dict, key=f1_dict.get)

    best_conf_matrix = (
        confusion_matrix_dict[best_th_f1][0],
        confusion_matrix_dict[best_th_f1][1],
        confusion_matrix_dict[best_th_f1][2],
        confusion_matrix_dict[best_th_f1][3],
    )
    best_f1 = f1_dict[best_th_f1]

    # Cover
    best_cover = cover_dict[best_th_f1]

    best_th_cover = max(cover_dict, key=cover_dict.get)
    max_cover = cover_dict[best_th_cover]

    if margin_list is not None:
        max_f1_margins_dict = {}
        max_th_f1_margins_dict = {}
        for margin in margin_list:
            curr_max_th_f1_margin = max(
                final_f1_margin_dict_fixed[margin],
                key=final_f1_margin_dict_fixed[margin].get,
            )
            max_th_f1_margins_dict[margin] = curr_max_th_f1_margin
            max_f1_margins_dict[margin] = final_f1_margin_dict_fixed[margin][
                curr_max_th_f1_margin
            ]
    else:
        max_f1_margins_dict, max_th_f1_margins_dict = None, None

    # Time to FA, detection delay
    best_time_to_FA = fp_delay_dict[best_th_f1]
    best_delay = delay_dict[best_th_f1]

    if verbose:
        print("AUC:", round(auc, 4) if auc is not None else auc)
        print(
            "Time to FA {}, delay detection {} for best-F1 threshold: {}".format(
                round(best_time_to_FA, 4), round(best_delay, 4), round(best_th_f1, 4)
            )
        )
        print(
            "TN {}, FP {}, FN {}, TP {} for best-F1 threshold: {}".format(
                best_conf_matrix[0],
                best_conf_matrix[1],
                best_conf_matrix[2],
                best_conf_matrix[3],
                round(best_th_f1, 4),
            )
        )
        print(
            "Max F1 {}: for best-F1 threshold {}".format(
                round(best_f1, 4), round(best_th_f1, 4)
            )
        )
        print(
            "COVER {}: for best-F1 threshold {}".format(
                round(best_cover, 4), round(best_th_f1, 4)
            )
        )

        print(
            "Max COVER {}: for threshold {}".format(
                round(cover_dict[max(cover_dict, key=cover_dict.get)], 4),
                round(max(cover_dict, key=cover_dict.get), 4),
            )
        )
        if margin_list is not None:
            for margin in margin_list:
                print(
                    "Max F1 with margin {}: {} for best threshold {}".format(
                        margin,
                        round(max_f1_margins_dict[margin], 4),
                        round(max_th_f1_margins_dict[margin], 4),
                    )
                )

    return (
        (
            best_th_f1,
            best_time_to_FA,
            best_delay,
            auc,
            best_conf_matrix,
            best_f1,
            best_cover,
            best_th_cover,
            max_cover,
        ),
        (max_th_f1_margins_dict, max_f1_margins_dict),
        delay_dict,
        fp_delay_dict,
    )

# Model with LogReg / XGBoost head

In [3]:
class PredHeadModel():
    def __init__(
        self, 
        ens_model,
        train_dataloader,
        test_dataloader,
        head_type="logreg",
        num_models=10,
        hidden_size=26,
    ):
        assert head_type in ["logreg", "xgboost", "mlp"], "Unknown head type"
        
        self.model = ens_model
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        
        self.loss = None
        
        if head_type == "logreg":
            self.head = LogisticRegression()
        else:
            self.head = XGBClassifier()
        
        self.train_ens_out = None
        self.train_labels = None
        self.head_fitted = False
        
    def get_train_embeddings(self):
        ens_out_bank, labels_bank = [], []
        for batch, labels in tqdm(self.train_dataloader):
            ens_preds = self.model.predict_all_models(batch)
            ens_out_bank.append(ens_preds)
            labels_bank.append(labels)
            
        train_ens_out = torch.cat(ens_out_bank, dim=1).permute(1, 2, 0).detach() # (dataset_size, seq_len, num_models)
        train_labels = torch.cat(labels_bank, dim=0)
        
        self.train_ens_out = train_ens_out
        self.train_labels = train_labels
    
        return train_ens_out, train_labels
    
    def fit_head(self, n_epochs=10, batch_size=64, accelarator="cpu", lr=1e-3):
        assert self.train_ens_out is not None, "Need to get embeddings"
        assert self.train_labels is not None, "Need to get labels"
        
        seq_num, seq_len, model_num = self.train_ens_out.shape
        train_ens_out = self.train_ens_out.reshape(-1, model_num)
        train_labels = self.train_labels.reshape(seq_num * seq_len)

        self.head.fit(train_ens_out, train_labels)              
        self.head_fitted = True
    
    def get_test_predictions_banks(self):
        assert self.head_fitted, "Need to fit the head"
        
        test_out_bank, test_out_head_bank, test_uncertainties_bank, test_labels_bank = [], [], [], []

        for test_batch, test_labels in tqdm(self.test_dataloader):
            preds_mean, preds_std = self.model.predict(test_batch)
            ens_preds = self.model.preds.detach()

            test_out_bank.append(preds_mean.detach())
            test_uncertainties_bank.append(preds_std.detach())
            test_labels_bank.append(test_labels)

            ens_preds = ens_preds.permute(1, 2, 0) # -> (batch_size, seq_len, model_num)
            preds_head_batch = []
            for ens_preds_for_seq in ens_preds:
                preds_head = self.head.predict(ens_preds_for_seq)
                preds_head_batch.append(preds_head)

            preds_head_batch = torch.from_numpy(np.stack(preds_head_batch))
            test_out_head_bank.append(preds_head_batch)
        
        return test_out_bank, test_out_head_bank, test_uncertainties_bank, test_labels_bank

# Experiments

## BCE for HAR

In [4]:
model_type = "seq2seq"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, 'r') as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type
args_config["num_workers"] = 4

args_config["loss_type"] = "bce"

args_config["learning"]["accelerator"] = 'cpu'
args_config["learning"]["devices"] = 1

train_dataset, test_dataset = datasets.CPDDatasets(experiments_name).get_dataset_()

train_dataloader = DataLoader(train_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False)
test_dataloader_shuffle = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True
)

In [23]:
ens_har_bce = EnsembleCPDModel(args_config, n_models=10)
ens_har_bce.load_models_list(
    f"saved_models/{args_config['loss_type']}/{experiments_name}/full_sample"
)

In [25]:
pred_head_model = PredHeadModel(
    ens_har_bce,
    train_dataloader,
    test_dataloader,
    head_type="logreg"
)

In [26]:
_ = pred_head_model.get_train_embeddings()
pred_head_model.fit_head()
(
    test_out_bank, test_out_head_bank, test_uncertainties_bank, test_labels_bank
) = pred_head_model.get_test_predictions_banks()

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [10]:
threshold_number = 100
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

evaluation_pipeline(    
    test_out_bank=test_out_bank,
    test_uncertainties_bank=test_uncertainties_bank,
    test_labels_bank=test_labels_bank,
    threshold_list=threshold_list,
    device="cpu",
    verbose=True,
    model_type="seq2seq",
    margin_list=[1, 2, 4, 8]
);

  0%|          | 0/102 [00:00<?, ?it/s]

AUC: 44.9517
Time to FA 11.0666, delay detection 0.095 for best-F1 threshold: 0.6117
TN 171, FP 7, FN 10, TP 1149 for best-F1 threshold: 0.6117
Max F1 0.9927: for best-F1 threshold 0.6117
COVER 0.9938: for best-F1 threshold 0.6117
Max COVER 0.994: for threshold 0.5875
Max F1 with margin 1: 0.9878 for best threshold 0.5875
Max F1 with margin 2: 0.99 for best threshold 0.5875
Max F1 with margin 4: 0.9922 for best threshold 0.5628
Max F1 with margin 8: 0.9931 for best threshold 0.5628


In [27]:
# head
evaluation_pipeline(    
    test_out_bank=test_out_head_bank,
    test_uncertainties_bank=test_uncertainties_bank,
    test_labels_bank=test_labels_bank,
    threshold_list=[0.5],
    device="cpu",
    verbose=True,
    model_type="seq2seq",
    margin_list=[1, 2, 4, 8]
);

  0%|          | 0/1 [00:00<?, ?it/s]

AUC: 0.0
Time to FA 10.9327, delay detection 0.0464 for best-F1 threshold: 0.5
TN 168, FP 26, FN 5, TP 1138 for best-F1 threshold: 0.5
Max F1 0.9866: for best-F1 threshold 0.5
COVER 0.9928: for best-F1 threshold 0.5
Max COVER 0.9928: for threshold 0.5
Max F1 with margin 1: 0.9866 for best threshold 0.5
Max F1 with margin 2: 0.987 for best threshold 0.5
Max F1 with margin 4: 0.9901 for best threshold 0.5
Max F1 with margin 8: 0.9905 for best threshold 0.5


## TS-CP for HAR

In [5]:
model_type = "tscp"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, 'r') as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type
args_config["num_workers"] = 4

args_config["learning"]["accelerator"] = 'cpu'
args_config["learning"]["devices"] = 1

train_dataset, test_dataset = datasets.CPDDatasets(experiments_name).get_dataset_()

train_dataloader = DataLoader(train_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False)
test_dataloader_shuffle = DataLoader(test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True)

In [6]:
ens_har_tscp = EnsembleCPDModel(args_config, n_models=10)
ens_har_tscp.load_models_list(
    f"saved_models/{model_type}/{experiments_name}/window_{args_config['model']['window']}"
)

In [7]:
pred_head_model = PredHeadModel(
    ens_har_tscp,
    train_dataloader,
    test_dataloader,
    head_type="logreg"
)

In [8]:
_ = pred_head_model.get_train_embeddings()
pred_head_model.fit_head()

(
    test_out_bank, test_out_head_bank, test_uncertainties_bank, test_labels_bank
) = pred_head_model.get_test_predictions_banks()

  0%|          | 0/448 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
threshold_number = 100
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

evaluation_pipeline(    
    test_out_bank=test_out_bank,
    test_uncertainties_bank=test_uncertainties_bank,
    test_labels_bank=test_labels_bank,
    threshold_list=threshold_list,
    device="cpu",
    verbose=True,
    model_type="seq2seq",
    margin_list=[1, 2, 4, 8]
);

  0%|          | 0/102 [00:00<?, ?it/s]

AUC: 48.0926
Time to FA 10.279, delay detection 0.8841 for best-F1 threshold: 0.7428
TN 156, FP 130, FN 76, TP 975 for best-F1 threshold: 0.7428
Max F1 0.9045: for best-F1 threshold 0.7428
COVER 0.9125: for best-F1 threshold 0.7428
Max COVER 0.9383: for threshold 0.3646
Max F1 with margin 1: 0.9304 for best threshold 0.3646
Max F1 with margin 2: 0.9437 for best threshold 0.2384
Max F1 with margin 4: 0.9489 for best threshold 0.2384
Max F1 with margin 8: 0.9494 for best threshold 0.2384


In [20]:
evaluation_pipeline(    
    test_out_bank=test_out_head_bank,
    test_uncertainties_bank=test_uncertainties_bank,
    test_labels_bank=test_labels_bank,
    threshold_list=[0.5],
    device="cpu",
    verbose=True,
    model_type="seq2seq",
    margin_list=[1, 2, 4, 8]
);

  0%|          | 0/1 [00:00<?, ?it/s]

AUC: 0.0
Time to FA 9.5744, delay detection 0.2468 for best-F1 threshold: 0.5
TN 145, FP 289, FN 12, TP 891 for best-F1 threshold: 0.5
Max F1 0.8555: for best-F1 threshold 0.5
COVER 0.9382: for best-F1 threshold 0.5
Max COVER 0.9382: for threshold 0.5
Max F1 with margin 1: 0.9193 for best threshold 0.5
Max F1 with margin 2: 0.9348 for best threshold 0.5
Max F1 with margin 4: 0.9447 for best threshold 0.5
Max F1 with margin 8: 0.947 for best threshold 0.5


## TS-CP for YAHOO

In [ ]:
# TODO

## BCE for Explosion

In [ ]:
# TODO

# MLP Head

In [16]:
class MLPHeadModel(pl.LightningModule):
    def __init__(
        self,
        ens_model,
        train_dl,
        val_dl,
        num_models=10,
        hidden_size=16,
        learning_rate: float = 1e-3,
    ) -> None:

        super().__init__()

        self.model = ens_model

        # freeze backbone model
        for m in self.model.models_list:
            for param in m.parameters():
                param.requires_grad = False

        self.head = nn.Sequential(
            nn.Linear(num_models, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid(),
        )
        # BCE loss for seq2seq binary classification
        self.loss = nn.BCELoss()

        self.lr = learning_rate
        
        self.train_dl = train_dl
        self.val_dl = val_dl

    def forward(self, inputs):
        ens_preds = self.model.predict_all_models(inputs)
        ens_preds = ens_preds.permute(1, 2, 0)
        out = self.head(ens_preds)
            
        return out.squeeze(-1)

    def training_step(self, batch, batch_idx) -> Dict[str, float]:
        inputs, labels = batch
        preds = self.forward(inputs)
        loss = self.loss(preds, labels.float())
        
        acc = ((preds > 0.5).long() == labels).float().mean()
        
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_epoch=True)
        
        return {"preds": preds, "loss": loss, "acc": acc}

    def validation_step(self, batch, batch_idx) -> Dict[str, float]:
        inputs, labels = batch
        preds = self.forward(inputs)
        loss = self.loss(preds, labels.float())
        
        acc = ((preds > 0.5).long() == labels).float().mean()

        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        
        return {"preds": preds, "loss": loss, "acc": acc}

    def configure_optimizers(self) -> torch.optim.Optimizer:
        """Initialize optimizer for the LocalValidationModel."""
        opt = torch.optim.Adam(self.head.parameters(), lr=self.lr)
        return opt
    
    def train_dataloader(self):
        return self.train_dl
        
    def val_dataloader(self):
        return self.val_dl 

In [17]:
model_type = "seq2seq"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, 'r') as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type
args_config["num_workers"] = 4

args_config["loss_type"] = "bce"

args_config["learning"]["accelerator"] = 'cpu'
args_config["learning"]["devices"] = 1

train_dataset, test_dataset = datasets.CPDDatasets(experiments_name).get_dataset_()

train_dataloader = DataLoader(train_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False)
test_dataloader_shuffle = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=True
)

In [18]:
ens_har_bce = EnsembleCPDModel(args_config, n_models=10)
ens_har_bce.load_models_list(
    f"saved_models/{args_config['loss_type']}/{experiments_name}/full_sample"
)

In [19]:
mlp_head_model = MLPHeadModel(
    ens_model=ens_har_bce,
    train_dl=train_dataloader,
    val_dl=test_dataloader,
    num_models=10,
    hidden_size=16,
    learning_rate=1e-3,
)

In [20]:
batch, labels = next(iter(train_dataloader))

out = mlp_head_model(batch)
out.shape

torch.Size([64, 20])

In [ ]:
comet_logger = CometLogger(
  
)

trainer = pl.Trainer(
  accelerator="cpu",
  max_epochs=10,
  logger=comet_logger
)

trainer.fit(mlp_head_model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/alex/Desktop/LARSS/CPD/code/UE_for_CPD_6.10/lightning_logs

  | Name | Type       | Params
------------------------------------
0 | head | Sequential | 193   
1 | loss | BCELoss    | 0     
------------------------------------
193       Trainable params
0         Non-trainable params
193       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
# TODO: check and evaluate